In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
%matplotlib inline

## Upload, Initial Exploration

In [3]:
df = pd.read_csv('application_data_updated.csv')

In [4]:
# Drop SK_ID_CURR column, not needed since we're not combining
df = df.drop('SK_ID_CURR', axis = 1)

In [5]:
print(df.shape)
df.head()

(307511, 51)


,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,2.0,2.0,2.0,-1134.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,1.0,0.0,-828.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,-815.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0.0,2.0,0.0,-617.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,-1106.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.dtypes

TARGET                           int64
NAME_CONTRACT_TYPE              object
CODE_GENDER                     object
FLAG_OWN_CAR                    object
FLAG_OWN_REALTY                 object
CNT_CHILDREN                     int64
AMT_INCOME_TOTAL               float64
AMT_CREDIT                     float64
AMT_ANNUITY                    float64
AMT_GOODS_PRICE                float64
NAME_TYPE_SUITE                 object
NAME_INCOME_TYPE                object
NAME_EDUCATION_TYPE             object
NAME_FAMILY_STATUS              object
NAME_HOUSING_TYPE               object
REGION_POPULATION_RELATIVE     float64
DAYS_BIRTH                       int64
DAYS_EMPLOYED                    int64
DAYS_REGISTRATION              float64
DAYS_ID_PUBLISH                  int64
FLAG_MOBIL                       int64
FLAG_EMP_PHONE                   int64
FLAG_WORK_PHONE                  int64
FLAG_CONT_MOBILE                 int64
FLAG_PHONE                       int64
FLAG_EMAIL               

## Cleaning + Dealing with Missing Data

In [7]:
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

AMT_ANNUITY                      12
AMT_GOODS_PRICE                 278
NAME_TYPE_SUITE                1292
OCCUPATION_TYPE               96391
CNT_FAM_MEMBERS                   2
EXT_SOURCE_2                    660
OBS_30_CNT_SOCIAL_CIRCLE       1021
DEF_30_CNT_SOCIAL_CIRCLE       1021
OBS_60_CNT_SOCIAL_CIRCLE       1021
DEF_60_CNT_SOCIAL_CIRCLE       1021
DAYS_LAST_PHONE_CHANGE            1
AMT_REQ_CREDIT_BUREAU_HOUR    41519
AMT_REQ_CREDIT_BUREAU_DAY     41519
AMT_REQ_CREDIT_BUREAU_WEEK    41519
AMT_REQ_CREDIT_BUREAU_MON     41519
AMT_REQ_CREDIT_BUREAU_QRT     41519
AMT_REQ_CREDIT_BUREAU_YEAR    41519
dtype: int64

In [8]:
# AMT_ANNUITY: drop if NA? hard to impute these, especially gien context of problem
df['AMT_ANNUITY'].value_counts()

9000.0     6385
13500.0    5514
6750.0     2279
10125.0    2035
37800.0    1602
           ... 
15210.0       1
50265.0       1
73012.5       1
40558.5       1
4437.0        1
Name: AMT_ANNUITY, Length: 13672, dtype: int64

In [9]:
# AMT_GOODS_PRICE: For consumer loans it is the price of the goods for which the loan is given
# Impute nulls with median, since the NA ones are those without consumer loans
df['AMT_GOODS_PRICE'].fillna((df['AMT_GOODS_PRICE'].median()), inplace=True)

In [10]:
# NAME_TYPE_SUITE: who was accompanying client when he was appplying for loan
# Change the null values to "Unaccompanied"
df['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace = True)

In [11]:
# CNT_FAM_MEMBERS: change nulls to 0
df['CNT_FAM_MEMBERS'].describe()
df.loc[df['CNT_FAM_MEMBERS'].isnull()] = 0

In [12]:
# DAYS_LAST_PHONE_CHANGE: change nulls to 0
df['DAYS_LAST_PHONE_CHANGE'].value_counts()
df.loc[df['DAYS_LAST_PHONE_CHANGE'].isnull()] = 0

In [13]:
# High correlation between 30 and 60 for social circle - just use 60
social_subset = df.loc[:, ['OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', \
                           'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']]
social_subset.corr()

df = df.drop(['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE'], axis = 1)

In [14]:
df.loc[df['OBS_60_CNT_SOCIAL_CIRCLE'].isnull()] = 0
df.loc[df['DEF_60_CNT_SOCIAL_CIRCLE'].isnull()] = 0

In [15]:
# Change amt_req_credit_bureau
df['AMT_REQ_CREDIT_BUREAU'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'] + df['AMT_REQ_CREDIT_BUREAU_DAY'] + \
                            df['AMT_REQ_CREDIT_BUREAU_WEEK'] + df['AMT_REQ_CREDIT_BUREAU_MON'] + \
                            df['AMT_REQ_CREDIT_BUREAU_QRT'] + df['AMT_REQ_CREDIT_BUREAU_YEAR']
# Remove old columns
df.drop(['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', \
         'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR'], axis = 1, inplace = True)

In [16]:
# Value counts are really skewed
df['AMT_REQ_CREDIT_BUREAU'].value_counts()
# Impute nulls with median, because of skewness (and this column doesn't seem that interesting)
df['AMT_REQ_CREDIT_BUREAU'].fillna((df['AMT_REQ_CREDIT_BUREAU'].median()), inplace=True)

In [17]:
# Check to see how many are null
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

AMT_ANNUITY           12
OCCUPATION_TYPE    96065
EXT_SOURCE_2         655
dtype: int64

In [18]:
# Final step: remove all rows without, since these are probably important
# Drop occupation too?
df = df.dropna(subset = ['AMT_ANNUITY', 'EXT_SOURCE_2'])
df.shape

(306844, 44)

In [19]:
# Check to make sure data types weren't messed up during the cleaning process
# df.dtypes

## EDA

In [20]:
# None really have that strong of a correlation with target...sad
df.corr()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU
TARGET,1.000000,0.019702,-0.002980,-0.028811,-0.010714,-0.038008,-0.035337,0.073029,-0.044568,0.040569,...,0.007334,0.003228,0.044714,0.051666,0.033141,-0.155909,0.009542,0.031467,0.054186,0.008295
CNT_CHILDREN,0.019702,1.000000,0.014437,0.005160,0.025156,0.001167,-0.022398,0.316967,-0.238642,0.180042,...,0.008454,0.015066,0.020644,0.071590,0.070817,-0.012784,0.016402,-0.001252,-0.007900,-0.034742
AMT_INCOME_TOTAL,-0.002980,0.014437,1.000000,0.158809,0.193636,0.161310,0.076972,0.018231,-0.063140,0.024251,...,0.062501,0.058151,0.004387,0.008055,0.009712,0.066066,-0.011647,-0.012478,-0.021706,0.025896
AMT_CREDIT,-0.028811,0.005160,0.158809,1.000000,0.771357,0.986828,0.106784,-0.071772,-0.064759,0.002377,...,0.053106,0.053747,-0.025374,-0.015986,0.002542,0.142571,0.003078,-0.022408,-0.079233,-0.002365
AMT_ANNUITY,-0.010714,0.025156,0.193636,0.771357,1.000000,0.775860,0.126852,-0.012804,-0.101591,0.029283,...,0.080586,0.075674,-0.004397,0.004479,0.013165,0.140231,-0.008069,-0.022219,-0.071035,0.018539
AMT_GOODS_PRICE,-0.038008,0.001167,0.161310,0.986828,0.775860,1.000000,0.110161,-0.069533,-0.062781,0.004431,...,0.054296,0.053912,-0.025707,-0.017470,0.001141,0.150190,0.003144,-0.023157,-0.081864,-0.003332
REGION_POPULATION_RELATIVE,-0.035337,-0.022398,0.076972,0.106784,0.126852,0.110161,1.000000,-0.047029,-0.001839,-0.060757,...,0.057541,0.081473,-0.048558,-0.040490,-0.012191,0.208781,-0.007639,0.003810,-0.050487,0.034603
DAYS_BIRTH,0.073029,0.316967,0.018231,-0.071772,-0.012804,-0.069533,-0.047029,1.000000,-0.607836,0.340681,...,0.090970,0.065537,0.172924,0.230485,0.149764,-0.120672,-0.000099,-0.001261,0.094673,-0.072484
DAYS_EMPLOYED,-0.044568,-0.238642,-0.063140,-0.064759,-0.101591,-0.062781,-0.001839,-0.607836,1.000000,-0.211668,...,-0.106642,-0.095039,-0.090059,-0.253556,-0.217269,-0.016671,0.006771,0.014506,0.021537,0.031399
DAYS_REGISTRATION,0.040569,0.180042,0.024251,0.002377,0.029283,0.004431,-0.060757,0.340681,-0.211668,1.000000,...,0.035661,0.026564,0.062900,0.096903,0.070160,-0.071502,0.006295,0.003242,0.062217,-0.028921


## Target Analysis, Preprocessing

In [21]:
# TARGET = our predicted value, 1 = default, 0 = no default
y = df['TARGET']
print(y.mean())
print(y.value_counts())

0.08061751248191264
0    282107
1     24737
Name: TARGET, dtype: int64


In [22]:
# Columns: Count_fam_members, income, credit, annuity, contract type, gender, education
X = df[['CNT_FAM_MEMBERS', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'CODE_GENDER', 'NAME_CONTRACT_TYPE', 
       'NAME_EDUCATION_TYPE']]

In [23]:
X.dtypes

CNT_FAM_MEMBERS        float64
AMT_INCOME_TOTAL       float64
AMT_CREDIT             float64
AMT_ANNUITY            float64
CODE_GENDER             object
NAME_CONTRACT_TYPE      object
NAME_EDUCATION_TYPE     object
dtype: object

In [28]:
X = pd.get_dummies(X)

In [29]:
X.dtypes

CNT_FAM_MEMBERS                                      float64
AMT_INCOME_TOTAL                                     float64
AMT_CREDIT                                           float64
AMT_ANNUITY                                          float64
CODE_GENDER_0                                          uint8
CODE_GENDER_F                                          uint8
CODE_GENDER_M                                          uint8
CODE_GENDER_XNA                                        uint8
NAME_CONTRACT_TYPE_0                                   uint8
NAME_CONTRACT_TYPE_Cash loans                          uint8
NAME_CONTRACT_TYPE_Revolving loans                     uint8
NAME_EDUCATION_TYPE_0                                  uint8
NAME_EDUCATION_TYPE_Academic degree                    uint8
NAME_EDUCATION_TYPE_Higher education                   uint8
NAME_EDUCATION_TYPE_Incomplete higher                  uint8
NAME_EDUCATION_TYPE_Lower secondary                    uint8
NAME_EDUCATION_TYPE_Seco

## Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)
# We set random_state=0 for reproducibility 
linear_classifier = SGDClassifier(random_state=0)

# Instantiate the GridSearchCV object and run the search
parameters = {'loss':['hinge', 'log'], 'penalty':['l1', 'l2']}
searcher = GridSearchCV(linear_classifier, parameters, cv=5)
searcher.fit(X_train, y_train)

# Report the best parameters and the corresponding score
print("Best CV params", searcher.best_params_)
print("Best CV accuracy", searcher.best_score_)
print("Test accuracy of best grid search hypers:", searcher.score(X_test, y_test))